In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD   
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('wordsA.pkl','wb'))
pickle.dump(classes,open('classesA.pkl','wb'))

# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
#training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = [item[0] for item in training]
train_y = [item[1] for item in training]
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=5, verbose=1)
model.save('chatbot_modelA.h5', hist)

print("model created")


[nltk_data] Downloading package punkt to C:\Users\SPIRO-
[nltk_data]     PYTHON1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\SPIRO-
[nltk_data]     PYTHON1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


322 documents
56 classes ['airline_amenities_by_class', 'airline_baggage_handling', 'airline_baggage_policy', 'airline_bereavement_fares', 'airline_booking_process', 'airline_cabin_classes', 'airline_cancellation_policies', 'airline_cancellation_policy', 'airline_check_in_options', 'airline_check_in_procedure', 'airline_check_in_process', 'airline_code_sharing', 'airline_complaint_handling', 'airline_customer_service_definition', 'airline_delay_compensation', 'airline_delayed_flight_compensation', 'airline_electronic_devices_policies', 'airline_emergency_exit_seat', 'airline_entertainment_options', 'airline_environmental_impact', 'airline_feedback_system', 'airline_flight_disruption_management', 'airline_flight_status_updates', 'airline_flight_upgrades', 'airline_food_allergy_policies', 'airline_frequent_flyer_program', 'airline_group_booking', 'airline_in_flight_meals', 'airline_infant_travel', 'airline_inflight_entertainment', 'airline_inflight_meals', 'airline_inflight_medical_emerg

C:\Users\SPIRO-PYTHON1\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
